In [1]:
!pip install tf-models-official
!pip install tf-nightly

     |████████████████████████████████| 696kB 3.1MB/s 
     |████████████████████████████████| 449.1MB 33kB/s 
     |████████████████████████████████| 3.9MB 36.9MB/s 
     |████████████████████████████████| 460kB 49.7MB/s 
     |████████████████████████████████| 2.9MB 49.7MB/s 
     |████████████████████████████████| 81kB 12.1MB/s 
  Created wheel for gast: filename=gast-0.3.2-cp36-none-any.whl size=9679 sha256=7ac4193518b4c13b5dbd4ea01db34072a44b3fca30bb44b6c3b2cc7eb2a4c50d
  Stored in directory: /root/.cache/pip/wheels/59/38/c6/234dc39b4f6951a0768fbc02d5b7207137a5b1d9094f0d54bf
Successfully built gast
ERROR: tensorflow 1.15.0 has requirement gast==0.2.2, but you'll have gast 0.3.2 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.10.1 which is incompatible.
ERROR: tb-nightly 2.2.0a20200106 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
  Found existing installation: google-auth 1.4.

In [0]:
import tensorflow as tf

In [2]:
tf.__version__

'2.2.0-dev20200115'

In [0]:
import tensorflow_hub as hub

from official.nlp.bert.tokenization import FullTokenizer
from official.nlp.bert.input_pipeline import create_squad_dataset
from official.nlp.bert.squad_lib import generate_tf_record_from_json_file

from official.nlp import optimization

from official.nlp.bert.squad_lib import read_squad_examples
from official.nlp.bert.squad_lib import FeatureWriter
from official.nlp.bert.squad_lib import convert_examples_to_features
from official.nlp.bert.squad_lib import write_predictions

In [0]:
import numpy as np
import math
import random
import time
import json
import collections
import os

from google.colab import drive

In [5]:
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
input_meta_data = generate_tf_record_from_json_file(
    "/content/drive/My Drive/ML/BERTSentimentAnalysis/train-v1.1.json",
    "/content/drive/My Drive/ML/BERTSentimentAnalysis/vocab.txt",
    "/content/drive/My Drive/ML/BERTSentimentAnalysis/train-v1.1.tf_record")

In [0]:
with tf.io.gfile.GFile("/content/drive/My Drive/ML/BERTSentimentAnalysis/train_meta_data", "w") as writer:
    writer.write(json.dumps(input_meta_data, indent=4) + "\n")

In [0]:
BATCH_SIZE = 4

train_dataset = create_squad_dataset(
    "/content/drive/My Drive/ML/BERTSentimentAnalysis/train-v1.1.tf_record",
    input_meta_data['max_seq_length'], # 384
    BATCH_SIZE,
    is_training=True)

In [12]:
input_meta_data

{'doc_stride': 128,
 'max_query_length': 64,
 'max_seq_length': 384,
 'task_type': 'bert_squad',
 'train_data_size': 88641,
 'version_2_with_negative': False}

In [11]:
next(iter(train_dataset))

({'input_ids': <tf.Tensor: shape=(4, 384), dtype=int32, numpy=
  array([[  101,  1996, 26129, ...,     0,     0,     0],
         [  101,  2129,  2116, ...,     0,     0,     0],
         [  101,  2000,  3183, ...,     0,     0,     0],
         [  101,  1999,  2054, ...,     0,     0,     0]], dtype=int32)>,
  'input_mask': <tf.Tensor: shape=(4, 384), dtype=int32, numpy=
  array([[1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>,
  'segment_ids': <tf.Tensor: shape=(4, 384), dtype=int32, numpy=
  array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>,
  'unique_ids': <tf.Tensor: shape=(4,), dtype=int32, numpy=array([1000000069, 1000000077, 1000000000, 1000000054], dtype=int32)>},
 {'end_positions': <tf.Tensor: shape=(4,), dtype=int32, numpy=array([ 38,  25, 137,  20], dtype=int32)>,
  'start_positions':

In [0]:
class BertSquadLayer(tf.keras.layers.Layer):

  def __init__(self):
    super(BertSquadLayer, self).__init__()
    self.final_dense = tf.keras.layers.Dense(
        units=2,
        kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02))

  def call(self, inputs):
    logits = self.final_dense(inputs) # (batch_size, seq_len, 2)

    logits = tf.transpose(logits, [2, 0, 1]) # (2, batch_size, seq_len)
    unstacked_logits = tf.unstack(logits, axis=0) # [(batch_size, seq_len), (batch_size, seq_len)] 
    return unstacked_logits[0], unstacked_logits[1]

In [0]:
class BERTSquad(tf.keras.Model):
    
    def __init__(self,
                 name="bert_squad"):
        super(BERTSquad, self).__init__(name=name)
        
        self.bert_layer = hub.KerasLayer(
            "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
            trainable=True)
        
        self.squad_layer = BertSquadLayer()
    
    def apply_bert(self, inputs):
        _ , sequence_output = self.bert_layer([inputs["input_ids"],
                                               inputs["input_mask"],
                                               inputs["segment_ids"]])
        return sequence_output

    def call(self, inputs):
        seq_output = self.apply_bert(inputs)

        start_logits, end_logits = self.squad_layer(seq_output)
        
        return start_logits, end_logits

In [0]:
TRAIN_DATA_SIZE = 88641
NB_BATCHES_TRAIN = 2000
BATCH_SIZE = 4
NB_EPOCHS = 3
INIT_LR = 5e-5
WARMUP_STEPS = int(NB_BATCHES_TRAIN * 0.1)

In [0]:
train_dataset_light = train_dataset.take(NB_BATCHES_TRAIN)

In [0]:
bert_squad = BERTSquad()

In [0]:
optimizer = optimization.create_optimizer(
    init_lr=INIT_LR,
    num_train_steps=NB_BATCHES_TRAIN,
    num_warmup_steps=WARMUP_STEPS)

In [0]:
def squad_loss_fn(labels, model_outputs):
    start_positions = labels['start_positions']
    end_positions = labels['end_positions']
    start_logits, end_logits = model_outputs

    start_loss = tf.keras.backend.sparse_categorical_crossentropy(
        start_positions, start_logits, from_logits=True)
    end_loss = tf.keras.backend.sparse_categorical_crossentropy(
        end_positions, end_logits, from_logits=True)
    
    total_loss = (tf.reduce_mean(start_loss) + tf.reduce_mean(end_loss)) / 2

    return total_loss

train_loss = tf.keras.metrics.Mean(name="train_loss")

In [0]:
bert_squad.compile(optimizer,
                   squad_loss_fn)

In [0]:
checkpoint_path = "/content/drive/My Drive/ML/BERTSentimentAnalysis/squad_ckpt"

ckpt = tf.train.Checkpoint(bert_squad=bert_squad)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Latest checkpoint restored!!")

In [22]:
train_dataset.take(1)

<TakeDataset shapes: ({input_ids: (4, 384), input_mask: (4, 384), segment_ids: (4, 384), unique_ids: (4,)}, {end_positions: (4,), start_positions: (4,)}), types: ({input_ids: tf.int32, input_mask: tf.int32, segment_ids: tf.int32, unique_ids: tf.int32}, {end_positions: tf.int32, start_positions: tf.int32})>

In [25]:
for (batch, (inputs, targets)) in enumerate(train_dataset_light):
  # print(batch)
  # print(inputs)
  print(targets)

{'end_positions': <tf.Tensor: shape=(4,), dtype=int32, numpy=array([ 36,  25, 124, 233], dtype=int32)>, 'start_positions': <tf.Tensor: shape=(4,), dtype=int32, numpy=array([ 33,  23, 123, 230], dtype=int32)>}
{'end_positions': <tf.Tensor: shape=(4,), dtype=int32, numpy=array([ 28, 111, 344, 101], dtype=int32)>, 'start_positions': <tf.Tensor: shape=(4,), dtype=int32, numpy=array([ 28, 109, 339,  95], dtype=int32)>}
{'end_positions': <tf.Tensor: shape=(4,), dtype=int32, numpy=array([135, 128,  63,  70], dtype=int32)>, 'start_positions': <tf.Tensor: shape=(4,), dtype=int32, numpy=array([133, 126,  58,  65], dtype=int32)>}
{'end_positions': <tf.Tensor: shape=(4,), dtype=int32, numpy=array([45, 33, 38, 57], dtype=int32)>, 'start_positions': <tf.Tensor: shape=(4,), dtype=int32, numpy=array([42, 30, 38, 57], dtype=int32)>}
{'end_positions': <tf.Tensor: shape=(4,), dtype=int32, numpy=array([122,  37,  44, 100], dtype=int32)>, 'start_positions': <tf.Tensor: shape=(4,), dtype=int32, numpy=array(

In [0]:
for epoch in range(NB_EPOCHS):
    print("Start of epoch {}".format(epoch+1))
    start = time.time()
    
    train_loss.reset_states()
    
    for (batch, (inputs, targets)) in enumerate(train_dataset_light):
        with tf.GradientTape() as tape:
            model_outputs = bert_squad(inputs)
            loss = squad_loss_fn(targets, model_outputs)
        
        gradients = tape.gradient(loss, bert_squad.trainable_variables)
        optimizer.apply_gradients(zip(gradients, bert_squad.trainable_variables))
        
        train_loss(loss)
        
        if batch % 50 == 0:
            print("Epoch {} Batch {} Loss {:.4f}".format(
                epoch+1, batch, train_loss.result()))
        
        if batch % 500 == 0:
            ckpt_save_path = ckpt_manager.save()
            print("Saving checkpoint for epoch {} at {}".format(epoch+1,
                                                                ckpt_save_path))
    print("Time taken for 1 epoch: {} secs\n".format(time.time() - start))

Start of epoch 1
Epoch 1 Batch 0 Loss 5.8978
Saving checkpoint for epoch 1 at /content/drive/My Drive/ML/BERTSentimentAnalysis/squad_ckpt/ckpt-1
Epoch 1 Batch 50 Loss 5.6089
Epoch 1 Batch 100 Loss 5.1082
Epoch 1 Batch 150 Loss 4.5536
Epoch 1 Batch 200 Loss 4.0513
Epoch 1 Batch 250 Loss 3.7111
Epoch 1 Batch 300 Loss 3.4320


In [0]:
RawResult = collections.namedtuple("RawResult",
                                   ["unique_id", "start_logits", "end_logits"])

In [0]:
def get_raw_results(predictions):
    for unique_ids, start_logits, end_logits in zip(predictions['unique_ids'],
                                                    predictions['start_logits'],
                                                    predictions['end_logits']):
        yield RawResult(
            unique_id=unique_ids.numpy(),
            start_logits=start_logits.numpy().tolist(),
            end_logits=end_logits.numpy().tolist())

In [0]:
all_results = []
for count, inputs in enumerate(eval_dataset):
    x, _ = inputs
    unique_ids = x.pop("unique_ids")
    start_logits, end_logits = bert_squad(x, training=False)
    output_dict = dict(
        unique_ids=unique_ids,
        start_logits=start_logits,
        end_logits=end_logits)
    for result in get_raw_results(output_dict):
        all_results.append(result)
    if count % 100 == 0:
        print("{}/{}".format(count, 2709))

In [0]:
output_prediction_file = "/content/drive/My Drive/projects/BERT/data/squad/predictions.json"
output_nbest_file = "/content/drive/My Drive/projects/BERT/data/squad/nbest_predictions.json"
output_null_log_odds_file = "/content/drive/My Drive/projects/BERT/data/squad/null_odds.json"

write_predictions(
    eval_examples,
    eval_features,
    all_results,
    20,
    30,
    True,
    output_prediction_file,
    output_nbest_file,
    output_null_log_odds_file,
    verbose=False)